In [1]:
# importing the required modules

import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import warnings
warnings.simplefilter("ignore")

In [2]:
# importing the required file

df = pd.read_csv("heart.csv")
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [3]:
# changing all the object value into integers

for col in df.columns:
  if df[col].dtype == "object":
    df[col] = df[col].astype("category")
    df[col] = df[col].cat.codes

In [4]:
# to check whether all the values changes to int/float or not

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  int8   
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  int8   
 3   AlcoholDrinking   319795 non-null  int8   
 4   Stroke            319795 non-null  int8   
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  int8   
 8   Sex               319795 non-null  int8   
 9   AgeCategory       319795 non-null  int8   
 10  Race              319795 non-null  int8   
 11  Diabetic          319795 non-null  int8   
 12  PhysicalActivity  319795 non-null  int8   
 13  GenHealth         319795 non-null  int8   
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  int8   
 16  KidneyDisease     31

In [5]:
# spliting data into x and y variables

x = df.drop(["HeartDisease"], axis= "columns")
y = df.pop("HeartDisease")

In [6]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: HeartDisease, dtype: int8

In [7]:
# further splitting data into x-train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size= 0.2, random_state= 7)

In [8]:
# making the input function

def input(features, labels, training= True, batch_size= 256):
  #converting the inputs into a dataset

  ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  #shuffle and repeat if you are in the training mode

  if training:
    ds = ds.shuffle(1000).repeat()

  return ds.batch(batch_size)

In [9]:
#feature column describe how to use the input function

my_feature_columns = []

for key in x_train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key= key))

In [10]:
# building the required prediction model

classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    hidden_units = [40, 20],                       # hidden layers containing 40 and 20 nodes
    n_classes = 2                                  # the resukt should in 2 different values
)

In [11]:
# training the model

classifier.train(
    input_fn = lambda: input(x_train, y_train, training= True),
    steps = 7000
)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
# finding the accuracy of the model

eval_result = classifier.evaluate(
    input_fn = lambda: input(x_test, y_test, training= False),
    steps= 7000
)

print("\nTest accuracy: {accuracy:0.3f}\n".format(**eval_result))


Test accuracy: 0.914

